In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r"data\processed_df_for_consort.csv")

In [3]:
exclude = ['נשירה מחקרית',
 'נשירה קלינית- לאחר ת. טיפול',
 'אין שת"פ טיפולי',
 'סיימו טיפול',
 'אי הסכמה למחקר',
 'אי התאמה למחקר',
 'פספוסי גיוסים']

In [4]:
N = {
    'isin':['CAU', 'IPC-SSC', 'אי הסכמה למחקר', 'אי התאמה למחקר', 'אין שת"פ טיפולי', 'משתתפים פעילים', 'נשירה מחקרית', 'נשירה קלינית- לאחר ת. טיפול', 'סיימו טיפול', 'עלייה לרמה 2', 'פספוסי גיוסים'],
    'not_in': []
}



Eligible = {
            'isin':['CAU', 'IPC-SSC', 'אי הסכמה למחקר', 'אין שת"פ טיפולי', 'משתתפים פעילים', 'נשירה מחקרית', 'נשירה קלינית- לאחר ת. טיפול', 'סיימו טיפול', 'עלייה לרמה 2'],
            'not_in': []
}



Randomized = {
            'isin':['CAU', 'IPC-SSC', 'אין שת"פ טיפולי', 'משתתפים פעילים', 'נשירה מחקרית', 'נשירה קלינית- לאחר ת. טיפול', 'סיימו טיפול', 'עלייה לרמה 2'],
           'not_in': []
}


Dropout = {
           'isin':['נשירה מחקרית', 'נשירה קלינית- לאחר ת. טיפול'],
            'not_in': []
                    }

research_dropout = {
                    'isin':['נשירה מחקרית'],
                    'not_in': []
                    }

clinical_dropout = {
                    'isin':['נשירה קלינית- לאחר ת. טיפול'],
                    'not_in': []
                    }


in_waiting_list = {
    'isin': ['משתתפים פעילים'],
    'not_in': ['CAU', 'IPC-SSC'] + exclude
}


Finished = {
    'isin': ['סיימו טיפול'],
    'not_in': []
}


Active = {
    'isin': ['משתתפים פעילים', 'CAU', 'IPC-SSC'],
    'not_in': exclude
}

not_cooperative = {
    'isin': ['אין שת"פ טיפולי'],
    'not_in': []
}

In [5]:
consort_groups = {
    
    'N': N,
    'Eligible': Eligible,
    'Randomized': Randomized,
    'Dropout': Dropout,
    'Research Dropout': research_dropout,
    'Clinical Dropout': clinical_dropout,
    'In Waiting List': in_waiting_list,
    'Finished': Finished,
    'Active': Active,
    'Not Cooperative': not_cooperative

}

In [6]:
# Extracting the detials of how to calc the groups 

# rows = []
# for i,j in consort_groups.items():
#     row = {
#         "consort_group": i,
#         "included_tables": j["isin"],
#         "excluded_tables": j["not_in"]
#     }
#     rows.append(row)

# pd.DataFrame(rows).to_csv("consort_groups_definition.csv", index=0)

In [7]:
def isin_consort_group(row, consort_group):
    in_pos_sheet = False
    for sheet in consort_group['isin']:
        if row[sheet]:
            in_pos_sheet = True

    in_neg_sheet = False
    for sheet in consort_group['not_in']:
        if row[sheet]:
            in_neg_sheet = True

    return (in_pos_sheet and (not in_neg_sheet))

In [8]:
for consort_calc in consort_groups.keys():
    df[consort_calc] = df.apply(isin_consort_group, axis=1, args = [consort_groups[consort_calc]])

In [9]:
for i in consort_groups.keys():
    print(i, df[i].sum())

N 301
Eligible 162
Randomized 129
Dropout 39
Research Dropout 27
Clinical Dropout 12
In Waiting List 38
Finished 30
Active 43
Not Cooperative 17


In [10]:
start = "first_contact_date"
end = 'therapy_starting_date'

In [11]:
df[start] = df[start].replace({'0': pd.NaT})
df[end] = df[end].replace({'0': pd.NaT})


In [12]:
df['waiting_duration'] =  pd.to_datetime(df[end]) - pd.to_datetime(df[start])
df['waiting_duration'] = df['waiting_duration'].dt.days

In [13]:
df['did_started_therapy'] = pd.notna(df[end])

In [20]:
df["first_contact_date"]

0      2025-03-16 00:00:00
1      2025-03-12 00:00:00
2      2025-01-07 00:00:00
3      2025-01-21 00:00:00
4      2025-04-20 00:00:00
              ...         
296    2024-09-23 00:00:00
297    2025-03-25 00:00:00
298    2024-12-16 00:00:00
299    2025-02-27 00:00:00
300    2025-06-01 00:00:00
Name: first_contact_date, Length: 301, dtype: object

In [14]:
# df.to_csv(r"data\export_with_days_delta.csv", index=0)
df.to_excel(r"data\export_with_days_delta.xlsx", index=0)

In [19]:
df.suiteable_for_pp == '1'

0      False
1       True
2       True
3       True
4       True
       ...  
296     True
297     True
298    False
299    False
300     True
Name: suiteable_for_pp, Length: 301, dtype: bool